In [15]:
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import shutil
import json

def path_link(path:str):
    sys.path.append(path)

path_link('/master/code/lib')

import dataLoading as dl
from measure import plotStdMessage
import simulation_v2 as sim
import features as ft



In [2]:
##############################
# paths 

#PATH = 'master/code/runs1'
#PATH = 'master/code/runs2'
#PATH = ['/master/code/analyze_models/exps/test_new_activation_0']
PATH = ['/master/code/analyze_models/exps/exp-test']

#DISPLAY_PATH = 'master/code/display_l1'
#DISPLAY_PATH = '/master/code/display_l1_2'
#DISPLAY_PATH = ['/master/code/analyze_models/display2/test_new_activation_0']
DISPLAY_PATH = ['/master/code/analyze_models/display-dec/exp-test']

MODEL_PATH = '/master/code/models/mod_base'


##############################

In [6]:
limits = [[0, 2], [2, 3], [3, 4], [4, 10]]
delta = 0.1

In [3]:
class Parameters():
    def __init__(self):
        self.dt = 0.001
        self.v0 = 60
        self.k = 70
        self.epsilon = 0.5
        self.tau = 3.5
        self.R = 1
        self.N = 200
        self.boundary = 100

        self.nbStep = 150

In [4]:
def findModels(path):
    pathLists = []
    for root, dirs, files in tqdm(os.walk(path)):
            for file in files:
                  
                  if file.endswith('.pt'):
                        pathLists.append(os.path.join(root, file))


    return pathLists


def delete_wandb_dirs(start_path):
    for root, dirs, files in os.walk(start_path, topdown=False):
        for dir_name in dirs:
            if dir_name == "wandb":
                dir_path = os.path.join(root, dir_name)
                print(f"Deleting: {dir_path}")
                shutil.rmtree(dir_path)



def getName(path):
    run_name = path.split('/')[-3]

    model_type = path.split('/')[-1].split('.')[0]

    if 'best' in model_type:
        model_type = '_best'

    else:
         model_type = '_latest'

    name = run_name + model_type

    return name


def loadModel(modelName:str, inputShape:int = 8, edges_shape = 5, path = None):
    """ 
    Function to import the model

    Args:
    -----
        - `modelName`: name of the model
        - `inputShape`: inout shape of the NN
        - `edges_shape`: edge shape of the NN
        - `path`: path where the models are
    """

    sys.path.append(path)

    loadFun = __import__(f'{modelName}', fromlist = ('loadNetwork'))

    model = loadFun.loadNetwork(inputShape, edges_shape)

    return model



def getModelName(key):

    name = ''

    #if 'simplest' in key:
    #    name = name + 'simplest'
    
    name = name + 'simplest'

    ## other possibilities

    if 'no-dropout' in key:
        name = name + '_no-dropout'
    else:
        name = name + '_dropout'

    if 'no-encoder' in key:
        name = name + '_no-encoder'
    else:
        name = name + '_encoder'

    if 'relu' in key:
        name = name + '-relu'

    return name


##############################
# Messages
#TODO linear 


def getOrderedVals(attribute, message, bins):
    bin_edges = np.linspace(np.min(attribute), np.max(attribute), bins + 1)
    bin_indices = np.digitize(attribute, bin_edges) - 1

    means = np.zeros(bins)
    stds = np.zeros(bins)

    for i in range(bins):
        bin_mask = bin_indices == i
        if np.any(bin_mask):
            means[i] = np.mean(message[bin_mask])
            stds[i] = np.std(message[bin_mask])
        else:
            means[i] = np.nan
            stds[i] = np.nan

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2


    return bin_centers, means, stds


def findIndices(message, nb = 5):
    stdv=plotStdMessage(message)
    plt.close()

    inds = np.argsort(stdv)
    # change of the order
    return np.flip(inds[-nb:])


def plotMessage(graph, messages, i_attr, id_message):
    edges = None
    
    for i in range(len(graph)):
        if edges is None:
            edges = graph[i].edge_attr.cpu().detach().numpy().copy()
        else:
            edges = np.vstack((edges, graph[i].edge_attr.cpu().detach().numpy().copy()))

    x, mean, std = getOrderedVals(edges[:, i_attr], messages[:, id_message], 100)
    plt.scatter(edges[:, i_attr], messages[:, id_message])


def calculate_interaction(dist, rij, k, epsilon, radii = 1.0):
    """
    Given the vectors ri and rj, compute the force between them
    """


    r = dist

    # Combined radius of both particles (assume unit radii for now)
    #bij = 2.0                       # Ri + Rj 
    bij = radii + radii

    if r < bij*(1 + epsilon):
        force = k*(r - bij)*rij/r  
    elif r < bij*(1 + 2*epsilon):
        force = -k*(r - bij - 2*epsilon*bij)*rij/r
    else:
        force = torch.tensor([0.0, 0.0])
    return force

In [5]:
def getData(params):
    N = params.N
    v0 = params.v0
    k = params.k
    eps = params.epsilon
    tau = params.tau
    R = params.R
    dt = params.dt
    nbStep = params.nbStep
    boundary = params.boundary

    lim = 0.85 * 100

    xPos = np.linspace(-lim, lim, 10)
    yPos = np.linspace(-lim, lim, 10)
    gridX, gridY = np.meshgrid(xPos, yPos)
    delta = np.random.uniform(0, 7, gridX.shape + (2,))

    gridX2 = gridX + delta[:, :, 0]
    gridY2 = gridY + delta[:, :, 1]

    pos = np.stack([gridX.ravel(), gridY.ravel()], axis=1)
    pos_perturbed = np.stack([gridX2.ravel(), gridY2.ravel()], axis=1)

    pos = np.concatenate([pos, pos_perturbed], axis=0)

    angles = np.random.rand(pos.shape[0]) * 2 * np.pi

    data = sim.compute_main(N, (v0, tau, k, eps), boundary, T = nbStep, initialization = (pos, angles), dt = dt)[0]

    x, y, attr, inds = ft.processSimulation(data)

    dataList = []

    for i in range(len(x)):
        g = Data(x = x[i][:, 2:], y = y[i], edge_attr = attr[i], edge_index = inds[i])
        dataList.append(g)

    return dataList

In [12]:
def getLoader(path, batch_size = 32, shuffleBool = True, root = None, jsonFile = None, mode = 'training'):
    datasetTraining = dl.DataLoader2(root, path = path, jsonFile = jsonFile, mode = mode)
    loader = DataLoader(datasetTraining, batch_size=batch_size, shuffle = shuffleBool)
    
    return loader


def findLoader(key):

    # remains initial conditions to consider

    if 'normal' in key:
        if '0_01' in key:
            dataloader_path = '/master/code/simulation/path/mew_0_01_normal.json'

        else:       # 0.001
            dataloader_path = '/master/code/simulation/path/mew_0_001_normal.json'

    else:       # noisy
        if '0_01' in key:
            dataloader_path = '/master/code/simulation/path/mew_0_01_noisy.json'

        else:       # 0.001
            dataloader_path = '/master/code/simulation/path/mew_0_001_noisy.json'


    print(f'>>>> HARDCODED loader')
    dataloader_path = '/master/code/simulation/path/mew_0_001_normal.json'

    print(dataloader_path)


    return getLoader(None, 128, True, None, dataloader_path, 'test')

In [40]:
def decomposeData(graph, messages, limits, delta, res = None):

    if res is None:
        res = {}

        for l in limits:

            res[f'{l[0]}-{l[1]}'] = {}
            res[f'{l[0]}-{l[1]}']['attr']= []
            res[f'{l[0]}-{l[1]}']['messages']= []
            res[f'{l[0]}-{l[1]}']['nodes']= []


    attr = graph.edge_attr
    inds = graph.edge_index
    nodes = graph.x

    for i in range(attr.shape[0]):
        dist = attr[i][0]

        for l in limits:
            l1 = l[0] + delta
            l2 = l[1] - delta
        

            if dist > l1 and dist < l2:
                res[f'{l[0]}-{l[1]}']['attr'].append(attr[i, :].cpu().detach().numpy().tolist())
                res[f'{l[0]}-{l[1]}']['messages'].append(messages[i, :].tolist())
                res[f'{l[0]}-{l[1]}']['nodes'].append(nodes[inds[0, i], :].cpu().detach().numpy().tolist())



    return res

    

In [41]:
def readJson(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def writeJson(data, filePath):
    """
    Function to write json 
    """
    with open(filePath, 'w') as f:
        json.dump(data, f, indent=2)

In [42]:
print(DISPLAY_PATH)

['/master/code/analyze_models/display-dec/exp-test']


In [56]:
def decomp_main(modelList, pathPlot = DISPLAY_PATH):

    number_messages=2
    nb_sim = 0

    with torch.no_grad():
        # get the data for the different models

        for model_path in tqdm(modelList):
            nb_sim += 1

            # path for the experiment
            p_exp = os.path.join(pathPlot, f'exp_{nb_sim}')
            if not os.path.exists(p_exp):
                os.makedirs(p_exp)
            else:
                print('WARNING: weird stuff here')


            name_plot = getName(model_path)


            #try:
            model = loadModel(getModelName(name_plot), path=MODEL_PATH)
            std_dict = torch.load(model_path)
            model.load_state_dict(std_dict)
            model.eval()

            # condition loader on the dt
            loader = findLoader(model_path)

            # get data
            for data, _ in loader:
                break

            print(data)

            # get messages
            message = model.message(data).cpu().detach().numpy()


            inds = findIndices(message, nb = number_messages)


            d = decomposeData(data, message[:, inds], limits, delta, res = None)

            writeJson(d, os.path.join(p_exp, 'data.json'))

            #except:
            #    print(f'Issue {nb_sim}')



In [57]:
if PATH is None:
    # check if listdir only outputs the last element (...)
    list_exp = [os.listdir('/master/code/analyze_models/exp/')]
    list_disp = [os.path.join('/master/code/analyze_models/display2', list_exp[i]) for i in range(len(list_exp))]

else:
    list_exp = PATH
    list_disp = DISPLAY_PATH


for i in range(len(list_exp)):

    exp = list_exp[i]
    disp = list_disp[i]

    model_list = findModels(exp)

4it [00:00, 32.34it/s]


In [58]:
print(model_list[0])

/master/code/analyze_models/exps/exp-test/test_new_activation/master-thesis_normal_classic_dt-0.01_lr-0.001_batch-16_encoder-no-encoder_dropout-no-dropout-exp/model_trained/simplest_no-dropout_no-encoder_aug_best_0-001-test.pt


In [59]:
decomp_main(model_list, pathPlot = DISPLAY_PATH[0])

  0%|          | 0/2 [00:00<?, ?it/s]

>>>> loading simplest
INFO >>> with NO encoder
INFO >>> with NO dropout


>>>> HARDCODED loader
/master/code/simulation/path/mew_0_001_normal.json
DataBatch(x=[38400, 10], edge_index=[2, 76536], edge_attr=[76536, 5], y=[38400, 10, 2], batch=[38400], ptr=[129])


 50%|█████     | 1/2 [00:18<00:18, 18.77s/it]

>>>> loading simplest
INFO >>> with NO encoder
INFO >>> with NO dropout
>>>> HARDCODED loader
/master/code/simulation/path/mew_0_001_normal.json
DataBatch(x=[38400, 10], edge_index=[2, 74710], edge_attr=[74710, 5], y=[38400, 10, 2], batch=[38400], ptr=[129])


100%|██████████| 2/2 [00:43<00:00, 21.76s/it]
